In [1]:
%%capture
#%pip install llama-index llama-index-embeddings-openai llama-index-embeddings-cohere==0.1.9 qdrant-client llama-index-vector-stores-qdrant llama-index-llms-openai llama-index-llms-cohere==0.1.19
%pip install llama-index mistralai llama-index-embeddings-mistralai qdrant-client llama-index-vector-stores-qdrant llama-index-llms-mistralai

In [3]:
import os
import sys
from getpass import getpass
import nest_asyncio

from IPython.display import Markdown, display

from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv()

sys.path.append('../helpers')

from utils import setup_llm, setup_embed_model, setup_vector_store

In [4]:
#OPENAI_API_KEY = os.environ['OPENAI_API_KEY'] or getpass("Enter your OPENAI_API_KEY key: ")
MISTRAL_API_KEY = os.environ['MISTRAL_API_KEY']

In [5]:
QDRANT_URL = os.environ['QDRANT_URL'] or getpass("Enter your Qdrant URL:")

In [6]:
QDRANT_API_KEY = os.environ['QDRANT_API_KEY'] or  getpass("Enter your Qdrant API Key:")

In [7]:
from llama_index.core.settings import Settings
from utils import setup_llm, setup_embed_model

setup_llm(
    provider="mistral", 
    model="mistral-small-latest", 
    api_key=MISTRAL_API_KEY, 
    temperature=0.75, 
    system_prompt="""Use ONLY the provided context and generate a complete, coherent answer to the user's query. 
    Your response must be grounded in the provided context and relevant to the essence of the user's query.
    """
    )

setup_embed_model(
    provider="mistral",
    api_key=MISTRAL_API_KEY
    )

In [8]:
from utils import get_documents_from_docstore
import random

all_documents = get_documents_from_docstore("../data/words-of-the-senpais")

random.seed(42)
senpai_documents = random.sample(all_documents, 100)

# 🧠 Semantic Chunking

This is a recent method that's been popularized by Greg Kamradt. He discussed this in detail in [an informative YouTube video](https://youtu.be/8OJC21T2SL4), which is also a great resource for more information on various chunking strategies.


Here's the gist of what semantic chunking does:

- Uses sentence embeddings to find breakpoints based on semantic similarity

- Keeps related sentences together in the same chunk

- Dynamically determines chunk size, no fixed length needed


## How semantic chunking works

1. ✂️ Split document into sentences

2. 🔢 Index sentences by position

3. 🎚️ Choose buffer size (sentences on either side to keep)

4. 📊 Measure similarity in embedding space
   - Keep similar sentences together
   - Split dissimilar sentences apart

5. 🧩 Merge groups based on similarity threshold


## [`SemanticSplitterNodeParser`](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/node_parser/text/semantic_splitter.py)

SemanticSplitterNodeParser a document into semantically related chunks called nodes. It uses semantic similarity and adaptive breakpoint determination, to create meaningful and coherent nodes from a document. 

### How it works

#### 1. 🧩 Document Splitting

   - The parser takes a document as input.

   - It splits the document into individual sentences using a sentence splitter (e.g., split_by_sentence_tokenizer).

#### 2. 🎚️ Sentence Grouping

   - The parser groups adjacent sentences together based on a configurable buffer size.

   - The buffer size determines how many sentences are considered together when evaluating semantic similarity.

   - For example, if the buffer size is 1, each sentence is treated individually. If it's greater than 1, sentences are grouped together.

#### 3. 🌐 Embedding Calculation

   - The parser calculates embeddings for each group of sentences using an embedding model.

   - Embeddings represent the semantic meaning of the sentence groups in a dense vector format.

#### 4. 📏 Distance Calculation

   - The parser calculates the cosine similarity between the embeddings of adjacent sentence groups.

   - It then computes the distance by subtracting the similarity from 1.

   - These distances represent the semantic dissimilarity between sentence groups.

#### 5. 🎯 Breakpoint Determination

   - The parser determines breakpoints based on a configurable percentile threshold (e.g., 95th percentile).

   - If the distance between two adjacent sentence groups exceeds the breakpoint threshold, it indicates a semantic shift and marks the start of a new node.

#### 6. 🧩 Node Creation

   - The parser splits the document into nodes based on the determined breakpoints.

   - Each node represents a semantically related chunk of text.

   - The sentences within a node are combined to form a coherent unit of information.

#### 7. 📝 Node Metadata

   - The parser can include additional metadata in the nodes, such as the original text or other relevant information.

   - It can also establish relationships between nodes, such as previous and next relationships, to maintain the sequential order of the chunks.

### Arguments you need to know

- `embed_model`: The embedding model to use for semantic comparison. If not provided, the parser will attempt to use the OpenAIEmbedding model. If the `llama-index-embeddings-openai llama-index-embeddings-cohere` package is not installed, an ImportError will be raised.

- `buffer_size`: The number of sentences to group together when evaluating semantic similarity. 

  - Default value is 1 (each sentence is considered individually). 
  
  - Increasing the buffer size allows the parser to consider the context of adjacent sentences when determining semantic similarity. This can help capture more meaningful relationships between sentences.

- `breakpoint_percentile_threshold`: The percentile of cosine dissimilarity that must be exceeded between a group of sentences and the next to form a node. 

  - A smaller value results in more nodes being generated. Default value is 95 (95th percentile). 

  - Adjusting this threshold allows you to control the granularity of the node splits. A lower value will create more nodes, while a higher value will create fewer nodes.

- `sentence_splitter`: The function or callable object used to split the text into sentences. Default is `split_by_sentence_tokenizer` (again, using the `PunktSentenceTokenizer` from the `nltk` library). The choice of sentence splitter can affect how the text is divided into individual sentences, which in turn influences the node splitting process.

  - While the `sentence_splitter` is used to initially split the document into individual sentences, the actual determination of node boundaries is based on semantic similarity rather than a fixed splitting approach.
  
  - `sentence_splitter` is more of a preprocessing step to prepare the document for the semantic analysis. It ensures that the parser has a consistent input format to work with (i.e., a list of sentences).

In [9]:
from llama_index.core.node_parser import SentenceSplitter, SemanticSplitterNodeParser

def semantic_splitter(
    embed_model,
    buffer_size, 
    breakpoint_percentile_threshold, 
    documents,
    **kwargs):
    splitter = SemanticSplitterNodeParser(
        embed_model=embed_model,
        buffer_size=buffer_size,
        breakpoint_percentile_threshold=breakpoint_percentile_threshold,
        )
    nodes = splitter.get_nodes_from_documents(documents)
    return nodes

## You have a lot of design choices to make here

These are all points of experimentation for you. Hack around with these, run evaluation against the metrics that matter to you, vibe check the results, and find one that works best.

- What embedding model do you want to use?

- What's the dimensions you want to use for the embedding model?

- What buffer size do you want to use? 

- What about the breakpoint threshold?

- What sentence splitter do you want to use?

I'll use some arbitrary settings for illustrative purposes.


In [11]:
semantic_nodes = semantic_splitter(
    embed_model = Settings.embed_model,
    buffer_size = 3, 
    breakpoint_percentile_threshold = 5, 
    documents = senpai_documents
    )

In [12]:
len(semantic_nodes)

1348

In [13]:
semantic_nodes[101].__dict__

{'id_': 'b8cc8072-563d-4847-ba0b-b5036fa4f540',
 'embedding': None,
 'metadata': {'page_number': 98,
  'file_name': '../data/hackers_and_painters.pdf',
  'title': 'Hackers and Painters',
  'author': 'Paul Graham'},
 'excluded_embed_metadata_keys': [],
 'excluded_llm_metadata_keys': [],
 'relationships': {<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b61d53d2-3936-4008-a295-646706222485', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_number': 98, 'file_name': '../data/hackers_and_painters.pdf', 'title': 'Hackers and Painters', 'author': 'Paul Graham'}, hash='6affca4b4a44bbe1fbdc2bd6b7dff1c38d239fe3ff8973c1c579ef52c48d671d'),
  <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='ce42deda-3e7f-406d-a721-b6a4787f8dde', node_type=<ObjectType.TEXT: '1'>, metadata={'page_number': 98, 'file_name': '../data/hackers_and_painters.pdf', 'title': 'Hackers and Painters', 'author': 'Paul Graham'}, hash='05a35e67fac5d43068bda2d2d70cb687c80ca108f7dcfad4ccfcb4612686abc1'),
  <N

In [14]:
print(semantic_nodes[100].get_content(metadata_mode="all"))

page_number: 98
file_name: ../data/hackers_and_painters.pdf
title: Hackers and Painters
author: Paul Graham

Its a much better deal for them to average their work together with a small group of their peers than to average it with everyone.


## 👷🏽‍♂️ 🗂️ Build the Index and Ingest to Qdrant

Note: This will also take a long time (about 30 minutes)


In [15]:
from llama_index.core import StorageContext
from llama_index.core.settings import Settings

from utils import create_index, create_query_engine, ingest, setup_vector_store

COLLECTION_NAME = "words-of-the-senpai-semantic-nodes"

semantic_nodes_vector_store = setup_vector_store(":memory:", QDRANT_API_KEY, COLLECTION_NAME)

Both client and aclient are provided. If using `:memory:` mode, the data between clients is not synced.


In [16]:
transforms = [Settings.embed_model]

semantic_nodes = ingest(
    documents=semantic_nodes,
    transformations=transforms,
    vector_store=semantic_nodes_vector_store
)

semantic_nodes_index = create_index(
    from_where="vector_store", 
    embed_model=Settings.embed_model,
    vector_store=semantic_nodes_vector_store
    )

### 🛠️ Setup Query Engine

In [17]:
from llama_index.core import PromptTemplate
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

from utils import create_query_engine
from prompts import HYPE_ANSWER_GEN_PROMPT

HYPE_ANSWER_GEN_PROMPT_TEMPLATE = PromptTemplate(HYPE_ANSWER_GEN_PROMPT)

semantic_nodes_query_engine = create_query_engine(
    index=semantic_nodes_index, 
    mode="query",
    response_mode="compact",
    similiarty_top_k=5,
    vector_store_query_mode="mmr", 
    vector_store_kwargs={"mmr_threshold": 0.42},
    text_qa_template=HYPE_ANSWER_GEN_PROMPT_TEMPLATE
    )

### 🔧 Setup Query Pipeline

In [18]:
from utils import create_query_pipeline

from llama_index.core.query_pipeline import InputComponent

input_component = InputComponent()

semantic_nodes_chain = [input_component,  semantic_nodes_query_engine]

semantic_nodes_query_pipeline = create_query_pipeline(semantic_nodes_chain)

In [19]:
semantic_nodes_query_pipeline.run(input="How can I navigate the maze of the market while building a company?")

> Running module 0e63c5bf-5790-4dd6-a671-50be1b343c2b with input: 
input: How can I navigate the maze of the market while building a company?

> Running module 2cb74527-4cfb-447b-b1dc-83db8c8a7afb with input: 
input: How can I navigate the maze of the market while building a company?



Response(response="Navigate the market maze by mapping out a complex decision tree. Diagram every possible alternative. Explain why your plan is superior to past failures and current competitors. Prove you have a unique idea. Sometimes, pitfalls only appear at scale. If this happens, re-enter the maze with a new strategy. You must be ready to adapt and pivot. This is how you build a company that stands out and wins. Don't be afraid to start over with a new approach if needed. The market is a battlefield, and you must be ready to fight and adapt.", source_nodes=[NodeWithScore(node=TextNode(id_='cf40983f-9e31-44da-9417-d38b98ee1039', embedding=None, metadata={'page_number': 159, 'file_name': '../data/anthology_of_balaji.pdf', 'title': 'The Anthology of Balaji Srinivasan', 'author': 'Balaji Srinivasan'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='31ccfef4-b875-4712-ba3d-d841c67e0d12', node_type='4